## Day 12 - Hot springs  
*(a return to this problem from scratch, one week after failing and giving up)*

Springs are:
- Operational (`.`)
- Damaged (`#`)
- Unkown (`?`)

In [33]:
with open("./example.txt") as f:
    example_lines = [line.strip() for line in f.readlines()]

with open("./input.txt") as f:
    input_lines = [line.strip() for line in f.readlines()]

example_lines

['???.### 1,1,3',
 '.??..??...?##. 1,1,3',
 '?#?#?#?#?#?#?#? 1,3,1,6',
 '????.#...#... 4,1,1',
 '????.######..#####. 1,6,5',
 '?###???????? 3,2,1']

In [34]:
def ways(string: str, rules: tuple[int]):
    if string == "":
        return 1 if rules == () else 0

    if rules == ():
        return 0 if "#" in string else 1

    result = 0

    # the string can either be a dot or # really (as ? just means either!)

    if string[0] in "?.":
        result += ways(string[1:], rules)
    
    if string[0] in "?#":
        if (
            rules[0] <= len(string)  # enough string to cover the first rule (we'll then move on from it!)
            and "." not in string[:rules[0]] # no . breaking up our chain of #s
            and (rules[0] == len(string) or string[rules[0]] in "?.") # and that one of 'the string is the length of the rule' (all #s) or that there isn't a # after the final # we need
        ):
            result += ways(string[rules[0] + 1:], rules[1:])


    return result

In [35]:
def part1(lines: list[str]):
    total = 0
    for line in lines:
        springs_str, rules_str = line.split()
        rules = tuple(map(int, rules_str.split(",")))
        total += ways(springs_str, rules)

    return total

assert part1(example_lines) == 21
part1(input_lines)

7653

clearly still inspired by mr Neutrino but I get it tbf

**Part 2: List of springs is actualy itself five times, separated by `?`**

In [45]:
def part2(lines: list[str]) -> int:

    cache = {}

    def ways(string: str, rules: tuple[int]):
        if string == "":
            return 1 if rules == () else 0

        if rules == ():
            return 0 if "#" in string else 1
        

        key = (string, rules)

        # Why can't I do this?
        # if result := cache.get(key, 0):
        #     return result

        if key in cache:
            return cache[key]
        
        result = 0


        # the string can either be a dot or # really (as ? just means either!)

        if string[0] in "?.":
            result += ways(string[1:], rules)
        
        if string[0] in "?#":
            if (
                rules[0] <= len(string)  # enough string to cover the first rule (we'll then move on from it!)
                and "." not in string[:rules[0]] # no . breaking up our chain of #s
                and (rules[0] == len(string) or string[rules[0]] in "?.") # and that the string is the length of the rule (all #s) or that there isn't a # after the final # we need
            ):
                result += ways(string[rules[0] + 1:], rules[1:])

        cache[key] = result

        return result

    total = 0
    for line in lines:
        springs_str, rules_str = line.split()
        rules_str = ",".join([rules_str for _ in range(5)])
        springs_str = "?".join([springs_str for _ in range(5)])
        rules = tuple(map(int, rules_str.split(",")))
        total += ways(springs_str, rules)

    return total

assert part2(example_lines) == 525152
part2(input_lines)

60681419004564